## 임베딩 읽어들이기

In [1]:
def load_word_embeddings(path, num_vecs=30000):
    embeddings, vocab = [], []
    f = open(path, "r", encoding="utf-8").readlines()[1:num_vecs + 1]
    for line in f:
        splited_line = line.strip().split(" ")
        word = splited_line[0]
        vec = [float(el) for el in splited_line[1:]]
        vocab.append(word)
        embeddings.append(vec)
    word_to_index = {word: idx for idx, word in enumerate(vocab)}
    index_to_word = {idx: word for idx, word in enumerate(vocab)}
    vocab_size = len(vocab)
    return embeddings, vocab, word_to_index, index_to_word, vocab_size

In [2]:
embeddings, vocab, word_to_index, index_to_word, vocab_size = load_word_embeddings("embedding/word2vec.txt")

## 관심 기능 정의

In [3]:
functions = ["배터리", "촬영", "디자인", "화면", "OS", "색"]
function_size = len(functions)

## 거리 행렬 구하기

In [4]:
import numpy as np
from scipy.spatial import distance

distance_matrix = np.zeros([function_size, vocab_size], dtype=np.float16)

In [5]:
for func_idx, func in enumerate(functions):
    func_vec = embeddings[word_to_index[func]]
    for word_idx, word_vec in enumerate(embeddings):
        distance_matrix[func_idx, word_idx] = distance.euclidean(func_vec, word_vec)

## 가중치 행렬 구하기

In [6]:
weight_matrix = np.zeros([function_size, vocab_size], dtype=np.float16)
for func_idx, per_func_distances in enumerate(distance_matrix):
    values = np.exp(-(per_func_distances ** 2) / 15)
    mean = np.mean(values)
    weight_matrix[func_idx, :] = values - mean + 0.5

## 가중치 행렬 시각화

In [7]:
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show

output_notebook()
def make_plot(title, hist, edges):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
    p.y_range.start = 0
    p.grid.grid_line_color="white"
    return p

Loading BokehJS ...

In [8]:
plots = []
for func_idx, func in enumerate(functions):
    hist, edges = np.histogram(weight_matrix[func_idx], density=True, bins=50)
    plots.append(make_plot(func, hist, edges))
show(gridplot(plots, ncols=2, plot_width=400, plot_height=300, toolbar_location=None))

In [13]:
battery_weights = [(token, score) for token, score in zip(vocab, weight_matrix[0])]

In [16]:
sorted(battery_weights, key=lambda x: x[1], reverse=True)

[('배터리', 1.058),
 ('밧데리', 0.928),
 ('터리', 0.7593),
 ('표준형', 0.7188),
 ('대용량', 0.718),
 ('대기전력', 0.693),
 ('리튬', 0.683),
 ('충전기', 0.6826),
 ('슬림형', 0.6753),
 ('급속', 0.6714),
 ('러닝', 0.6694),
 ('폴리머', 0.663),
 ('간당간당', 0.662),
 ('여분', 0.6553),
 ('발열', 0.653),
 ('임신', 0.6484),
 ('암페어', 0.647),
 ('배불뚝이', 0.647),
 ('런타임', 0.646),
 ('TTA', 0.645),
 ('상판', 0.644),
 ('기판', 0.6416),
 ('회로', 0.6416),
 ('외장형', 0.641),
 ('가로길이', 0.64),
 ('ISP', 0.639),
 ('RMAA', 0.635),
 ('커넥터', 0.633),
 ('접점', 0.633),
 ('소모율', 0.633),
 ('오래간다', 0.633),
 ('껍데기', 0.632),
 ('짹', 0.6313),
 ('단다고', 0.6304),
 ('mAH', 0.6304),
 ('개조', 0.63),
 ('나사', 0.629),
 ('서서히', 0.629),
 ('뽀대', 0.629),
 ('옹이', 0.629),
 ('보조', 0.6274),
 ('오링', 0.6274),
 ('비품', 0.6245),
 ('극심', 0.624),
 ('Ap', 0.624),
 ('넓이', 0.6235),
 ('절벽', 0.6235),
 ('하판', 0.623),
 ('스탠드', 0.6226),
 ('접지', 0.6226),
 ('타폰', 0.622),
 ('으다', 0.622),
 ('딸려온', 0.621),
 ('주변기기', 0.6206),
 ('전력', 0.62),
 ('변강', 0.6196),
 ('교실', 0.6196),
 ('커버도', 0.6196),
 ('aux', 0.6187),

## Document-Term Matrix 구축

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(vocabulary=vocab, binary=True, ngram_range=(1, 1), tokenizer=lambda x: x.strip().split())
corpus = open("data/phone_review_mecab.txt", "r", encoding="utf-8").readlines()
corpus = list(set([el for el in corpus if len(el.split()) < 31]))
dtm = vectorizer.fit_transform(corpus).toarray()
word_counts_per_doc = np.sum(dtm, axis=1)
result = np.dot(dtm, weight_matrix.T)

In [10]:
word_counts_per_doc

array([21,  4, 17, ..., 27, 15, 10])

In [11]:
word_counts_per_doc ** 0.6

array([6.21343239, 2.29739671, 5.47355332, ..., 7.22467406, 5.07755639,
       3.98107171])

## 최종 결과물 도출

In [12]:
for func_idx, func in enumerate(functions):
    print("##### " + func)
    scores = - result[:, func_idx] / (word_counts_per_doc ** 0.6)
    for sent_idx in scores.argsort()[:10]:
        print(corpus[sent_idx].strip(), - scores[sent_idx])
    print("\n")

##### 배터리
헐 저 도 그런데 완충 상태 표준형 끼워 놓 고 자 고 일어났 더니 5 0 가 줄 어 있 더라구요 근데 또 대용량 은 멀쩡 한데 배터리 문제 인가요 1.7026002081350753
X 6 하나 면 고성능 내장 라디오 Mp 3 카메라 통 품 1 6 G 메모리 변강 쇠 배터리 까지 3 5 인치 면 정말 좋 았 을 텐데 ㅠㅠ 1.6692199840833362
제 갤 s 3 배터리 두 개 중 하나 는 부풀 었 고 부푼 녀석 은 광탈 입니다 1 년 다 되 어 가 서 교환 도 안 되 죠 1.662229375491763
시리우스 그 두 개 외 에 도 배터리 표준형 슬림 이랑 대용량 있 어서 커버 2 개 가지 고 다녀야 되 는 게 너무 압박 이 네요 ㅎ ㄷ ㄷ 1.648775452332376
이 제품 은 배터리 커버 가 있 기 떄문에 주의 를 해야 합니다 실링 에 이 물질 이 끼 거나 덜 닫힌 상태 에서 담그 면 사망 하 는 겁니다 1.6430307078734288
그 럴리가요 저 는 미라크 유일 한 불만 이 배터리 인데 ㅎㅎ 아 그리고 윗 님 금오 충전기 통합 2 0 핀 으루 충전 하 심 꽤 빨라요 1.6424642177069533
5 3 인치 배터리 교체 가능 아몰 래드 Q 보이스 미 지원 CPU 스냅 3 세대 램 1 기 가 음 그냥 갤럭시 S 2 HD 가 화면 만 커진 1.6337532146840659
아몰 레기 가 배터리 효율 더 좋 지 않 나요 밝기 는 넘사벽 인데 갤 삼 밧데리 정말 오래 갑니다 IPS 나오 면 단점 도 있 어요 무거워 집니다 1.6329629499027059
갤 치디 는 스냅 3 세대 AP 를 사용 했으며 성능 과 배터리 면 에서 모두 엑시노스 4 2 1 0 에 밀립 니 다 1.6327360626676837
결국 노트 2 에 와서야 배터리 가 완전체 가 된 것 같 군요 노트 1 갤 삼 은 배터리 광탈 도 모자라 서 스웰 링 현상 까지 1.6176595059227625


##### 카메라
카메라 는 알파 부터 확실히 우위 였 

/Users/david/works/envs/py36tf18/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


갠 적 으로 노란색 보라색 파랑 색 이런 원색 좀 나오 지 옛 날 에 롤리팝 이나 쿠키 폰 이 색깔 은 굿 이 었 는데 ㅋㅋ 1.7173880624487896
블루 라이트 사용 하 니 완전히 검정 색 이 안 되 더라고요 약간 노란 빛 이 섞여서 그리고 켜 도 눈 이 침침 한 느낌 은 비슷 하 네요 ㅠㅠ 1.7054790394276322
그렇 긴 한데 뒷면 이 강렬 핫 핑크 라 남자 가 들 고 다니 기 엔 좀 그래요 뒷 케이스 만 다른 색 으로 바꾸 면 괜찮 을 득 1.701698145292929
아니 애초 에 둘 이 색깔 자체 가 다른데요 삼성 꺼 는 금괴 에 가까운 진한 골드색 이 고 아이폰 은 연한 샴페인 골드색 인데 단순히 골 드라고 1.697374489064111
레어 의 위엄 ㅋ 저 는 아이폰 4 이나 갤럭시 보다 요놈 디자인 이 이쁘 더라구요 ㅠ 근데 일반 AMOLED 라서 좀 망설여집니다 그 특유 의 색 번짐 이란 ㅠ 1.6882452260267915
선호도 는 블랙 인데 지금 산다면 화이트 요 ㅎㅎㅎ 기왕 사 는 것 더 신 제품 같 은 녀석 으로 사 야죠 핑크 는 고무장갑 색상 이 라서 논외 1.6860328484830456
우연히 실버 처럼 보이 게 찍힌 사진 인데 블루 에 요 ㅜ 근데 그냥 봐도 저렇게 실버 색 이 면 못 참 았 을 거 같 기 도 하 네요 1.6555749442226788
음 꽃 이나 웹툰 보 기 엔 색 이 진해서 어찌 보 면 괜찮 은 데 그래도 화이트 를 화이트로 표현 못 해 주 는 슬픈 액정 인 거 죠 1.6484663505093426
전 에 정가 주 고 검정 색 질렀었 는데 저렴 해서 흰색 으로 2 개 질렀 습니다 폰 이 흰색 이 니 색깔 맞 는 걸로 구입 했 네요 1.6459554393162723
배경 보 면 조명 색 인 거 같 은데요 근데 삼성 은 핑크 를 베이비핑크 보다 핫 핑크 스타일 로 내놓 던 데 나오 더라도 저런 베이비핑크 는 아닐 듯 1.6453914704379926




In [13]:
for func_idx, func in enumerate(functions):
    print("##### " + func)
    scores = - result[:, func_idx] / (word_counts_per_doc ** 0.6)
    i = 0
    for sent_idx in scores.argsort():
        if func not in corpus[sent_idx] and i < 10:
            print(corpus[sent_idx].strip(), - scores[sent_idx])
            i += 1
    print("\n")

##### 배터리
분명 아트 가 낳 지만 국산 어 플 구동력 이 딸리 죠 아크 는 오줌 액정 베가스 는 정체 가 뭐 지 베레기 답 은 뭐 다 갤 하 트 1.6059965068373339
전 헬레 이 서 인대 밧데리 가 조루 인 거 빼 건 성능 은 쓸 만 합니다 뒤 껍데기 2 개 나 주 고 액 보 까지 ㅋ 1.5999113318168585
테티 이랑 베가스 랑 같이 쓰 는 중 인데 인터넷 이랑 속도 는 발군 메모리 랑 어 플 저장 공간 은 어휴 ㅡㅡ 베 터리 도 오래가 고 1.5985496781406987
미친 듯 한 발열 3 G 가끔 끊김 박 대리 조기 퇴근 충전 속 도 드럽 게 느림 유격 홈 버튼 빛 샘 카메라 뻘건 멍 등등 1.585291143035531
카메라 는 알파 부터 확실히 우위 였 죠 센서 크롭 ois 의 부제 만 빼 면 노트 4 와 거의 같 은 스펙 에 품질 도 1.583770214809689
거치대 가 충전 도 되 지만 HDMI 출력 을 2 만 원 짜리 프라다 MHL 커넥터 ㄷ ㄷ 저 R 3 쓰 는데 호환 만 된다면 사 고 싶 은데요 1.5825382767214726
제조 업체 3 사 의 하이엔드 G 2 갤 4 ltea 베티 아 베 시업 스펙 은 동일 합니다 디자인 이나 액정 이나 ui 측면 에서 호불호 가 갈릴 뿐 1.5803546074445
제 가 쓰 고 있 는데 롤리팝 떄나 마시멜로 우 떄나 거의 완전 체급 입니다 4 3 비 와 충전 속도 뺴고는 단점 이 없 는 기계 입니다 1.5789084323720892
시리우스 와 베가 는 디자인 뎀 비 안테나 3 5 단자 감압 식 정전식 차이 로 알 고 있 어요 저 도 베 가 신규 기 달림 ㅠㅠ 1.5780416779074553
내장 메모리 를 늘려서 스냅드래곤 을 카바 치 는 건가 밧데리 2 개 랑 내장 메모리 용량 정도 가 해외 판과 의 경쟁력 은 아니 겠 죠 1.5762188203389103


/Users/david/works/envs/py36tf18/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until




##### 카메라
다시 와이드 비율 로 돌아왔 고 해상도 가 올라갔 네요 근데 스테레오 스피커 한쪽 에 몰려 있 는 건 진짜 멍청 하 고 쓸모없 는 구조 같 은데 대체 왜 1.6757104492060353
그렇 군요 대각선 에서 보 면 갤럭시 시리즈 는 다 스머프 고 심지어 프라다 도 시야각 저질 이 라던데 8 0 0 니트 의 노바 디스플레이 라도 1.6106175426579885
위상차 안 박힌 저 가 모델 들 은 사실 상 af 없 는 거 죠 뚜 잇 렌즈 에 조금 만 조도 떨어지 면 초점 못 잡 습니다 ㅋ CLiOS 1.5953763126141047
아니 애초 에 둘 이 색깔 자체 가 다른데요 삼성 꺼 는 금괴 에 가까운 진한 골드색 이 고 아이폰 은 연한 샴페인 골드색 인데 단순히 골 드라고 1.5914513561377701
분명 아트 가 낳 지만 국산 어 플 구동력 이 딸리 죠 아크 는 오줌 액정 베가스 는 정체 가 뭐 지 베레기 답 은 뭐 다 갤 하 트 1.5829751841461808
네이버 에서 보 니까 프로세서 1 4 GHz ARM Cortex A 9 듀얼 코어 Mali 4 0 0 MP GPU Exynos 칩셋 엑시노스 로 나오 는 거 같 네요 ㅎ 1.5811165579788053
3 7 Inch WVGA 4 8 0 X 8 0 0 6 5 K COLOR AMOLED Plus 이게 맞 는 스펙 인지요 1 6 0 0 만 컬러 가 아닌지 1.575988562233738
제 상전 도 아니 고 한낱 물건 따위 를 부르 는데 무슨 용기 씩 이나 아무리 뭐 라고 해도 화이트 노이즈 등 음감 성능 떨어지 는 건 사실 이 죠 1.5732646215993635
동영상 자주 보 시 면 옵 lte 가 좋 아요 정말 재 생력 발군 입니다 1 0 8 0 pmkv 확장자 무손실 flac 음성 인 영상 도 가볍 게 돌립니다 1.5714579504378365
방금 A 4 용지 조그맣 게 양면테이프 와 함께 오려서 근접 센서 맞닿 는 부위 에 붙였 더니 그런 현상 없 어 졌 네